# Investigation of Python data visualization tools

**Neil Vaytet, Piotr Rozyczko, Simon Ward, Andrew Sazonov**

*Data Reduction Analysis & Modeling group, Data Management and Software Center, the European Spallation Source, Copenhagen, Denmark*

Contact: neil.vaytet@esss.se

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/ca/ESS_Logo_Frugal_Blue_cmyk.png/320px-ESS_Logo_Frugal_Blue_cmyk.png" />

## Table of contents

* [Abstract](#abstract)
* [Introduction](#introduction)


* [Matplotlib](#matplotlib)
* [Plotly](#plotly)
* [PyQtGraph](#pyqtgraph)
* [Bqplot](#bqplot)
* [Bokeh](#bokeh)
* [HoloViz/hvPlot/Holoviews](#holoviz)
* [Ipyvolume](#ipyvolume)
* [Molecular/atomic viewers](#molecules)
* [Other projects](#other) (Vispy, Altair, pyvista, itkwidgets)


* [Conclusions](#conclusions)

<a id='abstract'></a>
## Abstract

We explored the Python data visualization ecosystem by selecting the most commonly used open-source libraries and testing them in a set of 10 standard [use cases](examples/matplotlib.ipynb). The packages were evaluated on their richness of features, capabilities for interaction (primarily in a Jupyter notebook environment), project sustainability, documentation, and performance.

Our findings are best represented as a ranked list:

1. [`plotly`](#plotly): best use case coverage (including 3D), excellent interactivity, great performance, has a `Dash` platform for creating in-browser apps. Coupling with `Datashader` could be a winner.


2. [`HoloViz`](#holoviz): very similar to `plotly` in case coverage, very high performance (especially via the `Datashader` method), makes interactions much easier to implement than `bokeh`, also has a `Panels` utility for apps, but is a young project that still feels scattered over different sub-projects, not as unified as `plotly`.


3. [`matplotlib`](#matplotlib): most mature and well-established project, largest community/user base, great case coverage, but 3D performance is poor, and setting up interactions is cumbersome. Should be complemented by a 3D-specific library (`ipyvolume` is a good candidate).


4. [`bokeh`](#bokeh): good case coverage, many interactions possibilities, but lacking 3D visualization, and interactions with buttons/sliders must be implemented in javascript.


5. [`pyqtgraph`](#pyqtgraph): outstanding performance, 1 to 3D under a unified interface, wide range of interactions, but only one developer, project feels un-finished (especially 3D graphs), and integration in Jupyter notebooks is poor.


6. [`bqplot`](#bqplot): focuses on interactions (every item in a plot is a clickable/draggable widget), the diversity of which, unfortunately, never makes up for the sad performance. It would be worth re-visiting this young project in a year's time.

---

<a id='introduction'></a>
## Introduction

In recent years, Python has established itself as one of the go-to languages for data visualization, most probably because of the speed of development it allows, as well as the open-source nature/spirit of many Python projects out there.

Incidently, this also meant that many different visualization libraries were born in a short amount of time, covering different use-cases, often with a significant amount of overlap between projects. This is best illustrated by [Jake VanderPlas’s](https://twitter.com/jakevdp?lang=en) overview of how the many different visualization libraries in Python currently relate to each other:

![python-vis-landscape](https://www.anaconda.com/wp-content/uploads/2019/01/PythonVisLandscape.jpg)

This figure is nicely complemented by [this](https://wiki.python.org/moin/NumericAndScientific/Plotting) almost complete list of packages.

The obvious question that comes immediately after being confronted with such a diversity of libraries is how does one make a choice when starting a new project that requires a data visualization component?

There are a number of investigations that explore the different options and their benefits/drawbacks, including a very useful 3-part post on the [Anaconda website](https://www.anaconda.com/python-data-visualization-2018-why-so-many-libraries/), and the background discussion on the HoloViz [website](https://holoviz.org/background.html). The Mantid developers also carried out their own investigation, and this is available upon request (via email to the authors).

These reviews however still felt too superficial for our needs at the Data Management and Software Center (DMSC) for the European Spallation Source (ESS). We are building an entire suite of applications for neutron data aquisition, reduction and analysis, with several completely new packages being developed. Our goal was to see if we could select a single library, or alternatively a small number of packages among the best available, that would cover all our visualization needs, from 1 to 3 dimensions, with strong interactive capabilities. Eventually, we would attempt to wrap this subset of libraries under a common API, that could be used across the entire DMSC software suite.

Before a choice could be made, it seemed essential to really drill down into each package and not only test its features and performance, but also survey the longevity of the project and the state of the userbase/community. We have several criteria that strongly influence our assessment of the different packages:

- Good integration in Jupyter notebooks is essential
- Performance is crucial as the ESS data rates will be higher than ever before
- Project sustainability is key, since ESS will operate for at least the next 30 years
- Ease of installation on all platforms is a must (Linux, OSX, Windows)
- Libraries must be well documented

This report is the result of our investigation into the most commonly-used packages, their pros and cons, and the use cases they cover. For most libraries, we have tried to reproduce a set of standard plotting needs, and these are located in the [examples](examples) directory. There are also links to the relevant notebooks in each of the following sections.

<a id='matplotlib'></a>
## Matplotlib (https://matplotlib.org/3.1.1/index.html)

`matplotlib` is by far the most popular and widely used package in the scientific data visualization community. Supported by NumFOCUS, it is an old (started in 2003) and very mature project, and yet still sees very active development.
It is the go-to library for creating publishable vector graphics pdf figures in articles, and specialises in 2D plots.

It is either already installed on most systems, or very easy to install on all platforms. Its strength is on high-quality static figures, and is behind other projects on the interactivity side of things.

It has some 3D capabilities through the `Axes3D`.

To see what `matplotlib` can do, a good place to go is [its sample plots page](https://matplotlib.org/tutorials/introductory/sample_plots.html).

A small example is shown below, to illustrate both the syntax and the output generated.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
%matplotlib notebook

In [ ]:
N = 50
x = np.arange(N)
y = np.arange(N)
z = np.random.rand(N, N)

fig, ax = plt.subplots(2, 2)
ax[0][0].plot(x, z[:, 0])
ax[0][0].set_title("1D line plot")
ax[0][1].hist(z[:, 0], bins=20)
ax[0][1].set_title("1D histogram")
ax[1][0].imshow(z)
ax[1][0].set_title("2D image")
ax[1][1].scatter(z[:, 0], z[:, 1], c=z[:, 2])
ax[1][1].set_title("Scatter plot")
fig.show()

### Interface

When talking about `matplotlib`, a common complaint that arises is that the API is a little difficult to get around, especially for a non-experienced user/programmer. This is somewhat true, but in some sense a little inevitable in a highly customizable tool. It is however relatively straightforward to hide the interface from the user by creating pre-made wrapper functions that will cover 90% of the cases the users are after.

Examples of this are very common, for example the `pandas` library uses `matplotlib` for creating plots from the contents of its data frames, via a simple `.plot` method.

### Interactivity

Another common misconception is that interactivity is virtually not supported in `matplotlib`. In fact, the library allows multiple interactivity options with mouse events (scrolling, clicking), as well as via its own set of `widgets`. These include sliders, radio buttons, check boxes, etc... and are a nice alternative to `ipywidgets`, guaranteed to work out-of-the-box with matplotlib figures.

See a [diagram](https://matplotlib.org/3.1.0/_images/inheritance-9d71a95f6d3ec40d1246549117ad7959f7b88c66.png) listing all the available widgets.

Interactivity is available both in the default Qt render window, and in the Jupyter notebook. Note that for the latter, a magic command `%matplotlib notebook` needs to be added **in a separate cell** at the start of the notebook, otherwise only static images are produced. Hiding this cell from the notebook inside a library import (to save the user from having to enter it in every new notebook) is apparently not yet possible.

[A few noteable examples of interactive plots](examples/matplotlib.ipynb#interactive_examples)

One has to note that, as illustrated by these additional examples, setting up interactivity in matplotlib takes more lines of code than in other projects, and there is definitely room for improvement in the interface.

### Performance

`matplotlib` is very fast, especially for generating images with regularly-sized pixels via its `imshow` function which is much faster than other web-based solutions listed in this report.

Saving figures to vector graphics (pdf) can however sometimes be a little slow.

Interactivity with plots shows mixed performance. Zooming onto lines or images is fast, but interaction with widgets feels sluggish. It has to be noted that interaction is smoother inside the `Qt` window compared to the Jupyter notebook. Maybe some emulation is taking place to render in the notebook, thus degrading performance?

Performance on 3D graphs is poor. Rotating a simple 3D surface outputs at something close to 1-2 FPS:

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

X = np.arange(-5, 5, 0.25)
X, Y = np.meshgrid(X, X)
Z = np.sin(np.sqrt(X**2 + Y**2))

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
surf = ax.plot_surface(X, Y, Z, linewidth=0, antialiased=True, cmap='viridis')
fig.colorbar(surf, shrink=0.5, aspect=5)
fig.show()

### Documentation
The documentation is very extensive, well written, with a complete API reference and examples galore.

### Community
The user base of `matplotlib` is unrivaled, and the variety of examples found around the web is the richest out there. T

### Risks and project sustainability

![mpl pulse](images/pulse_matplotlib.png)

The project has been around for many years, and will still be there for many to come. No real risks are associated with such a well established library. 

### Standard use cases
See [here](examples/matplotlib.ipynb) for a notebook demonstrating how some standard use cases are achieved using `matplotlib`.

### Summary

<span style="color:green">**Pros:**</span>

- Very mature project, used by the majority of scientific community for publication-grade figures
- Widespread availability on existing systems
- Easily installable
- Well-maintained with many developers
- Good 1D-2D performance (`imshow` is very fast for images with regularly-sized pixels)

<span style="color:red">**Cons:**</span>

- Poor 3D performance
- Interface to interactivity could be improved/made simpler

---

<a id='plotly'></a>
## Plotly (https://plot.ly/python/)

`plotly` is a very popular solution for creating highly interactive plots to be used in a web-browser. It is Javascript based, making use of the D3.js library, but also has a complete Python API, making it well suited for Jupyter notebook use.

It boasts a high performance, especially for scatter plots, by making use of the WebGL technology (see [here](https://plot.ly/python/webgl-vs-svg/)).

One very nice feature of `plotly` is that its output is standalone html code that can be embedded in any web-site.

The same example as for `matplotlib` is shown below

In [ ]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [ ]:
fig = make_subplots(rows=2, cols=2)

fig.add_trace(go.Scatter(x=x, y=z[:, 0]), row=1, col=1)
fig.add_trace(go.Histogram(x=z[:, 0], xbins={"size": 0.05}), row=1, col=2)
fig.add_trace(go.Heatmap(z=z, showscale=False), row=2, col=1)
fig.add_trace(go.Scatter(x=z[:, 0], y=z[:, 1], mode='markers',
    marker={"color": z[:, 2]}), row=2, col=2)

fig.show()

### Dash

The makers of `plotly` also offer a platform called `Dash` which aims to make it very easy to create web apps with interactive menus, buttons, sliders and plots. Examples can be found here: https://dash-gallery.plotly.host/Portal/, and some are indeed very impressive.

It is also now possible to embed `Dash` apps as a separate window in the JupyterLab environment, to allow both clickable and scriptable interaction with an app: https://github.com/plotly/jupyterlab-dash

This could be a very nice alternative to developing GUIs for something like interfaces in `Mantid`.

### Interactivity

Interactivity is varied and extremely well supported in `plotly`. It is one of the strengths of the library, and it feels like much work has gone into making this their selling point.

Zooming and panning is smooth, and connecting graphs to Jupyter `ipywidgets` works well. It is even possible to go further with selection events, whereby selecting points in a plot can affect what data is displayed in another plot or a table. See [here](https://plot.ly/python/selection-events/) for an exmaple.

### Performance

`plotly`'s performance is very good for line/scatter plots, even for very large datasets (1M points), via the `Scattergl` method.

Performance for 3D visualization seems reasonable, and far better than `matplotlib`.

### Documentation

Documentation is very rich, well written, with many examples. A full API reference is also available. A minor downside is that it is based around a search box which can sometimes feel quite slow.

### Community
A large community uses `plotly`, which means that many examples are available online to  get ideas from.

Developers are responsive to posts on the forum, and seem very happy to help.

### Risks and project sustainability

![plotly pulse](images/pulse_plotly.png)

From the `Github` pulse, it appears that the project has had switching lead developers, who have not had much overlap. Not much can be read from this, apart from the fact that it appears the project does get picked up if one of the main developers moves on to new things.

`plotly` has now been around for 5+ years, and the overall impression is that it is here to last. Out of the many available javascipt plotting libraries out there, it has established itself as one of the front-runners.

`plotly` is a company in its own rights, and secures funding from venture capitalists. 

### Standard use cases
See [here](examples/plotly.ipynb) for a notebook with the `plotly` standard use cases.

### Summary

<span style="color:green">**Pros:**</span>

- Interactivity &starf;&starf;&starf;&starf;&starf;
- Large well-maintained and well-documented project
- Very complete in terms of features
- 3D capable
- The `Dash` framework for building apps is very interesting

<span style="color:red">**Cons:**</span>

- Not easily useable outside of a web environment. It is however possible to use a custom renderer and open a `plotly` graph inside a Qt window (see [here](examples/plotly.ipynb#plotly_in_qt); note that in this example, the figure is first written to disk and then loaded by the Qt html window, which is not optimal)

---

<a id='pyqtgraph'></a>
## PyQtGraph (http://www.pyqtgraph.org/)

`pyqtgraph` is a very impressive tool based on OpenGL that provides 1D-3D data visualization with astonishing performance. Zooming in on a line made up of millions of points is flawless, and the same goes for a 5000x5000 image.

The OpenGL shaders can be used to create beautiful effects in 3D environments.

The idea to use OpenGL for 1, 2, 3D plots is quite brilliant, and in principle offers a completely seamless experience across all dimensions, thus solving a common problem where plotting libraries are often either good at 1-2D plotting, or 3D, but not all at the same time (cf `matplotlib` for example).

The installation is simple via `pip` or `conda` and comes with a large set of useful examples.

The rendered graphs do feel a little more 'raw' than other projects where more resources have been spent on aesthetics. That said, it is possible to make `pyqtgraph` more eye-friendly by tweaking background and foreground colors and styles. This however all needs to be done manually. 

### Interactivity

Interaction with graphs via mouse events is extremely smooth, even for large datasets. It is also possible to define regions-of-interest which can be moved/reshaped with the mouse.

As the name suggests, `pyqtgraph` integrates very well in `Qt` windows, and it is relatively simple to connect graphs to sliders, buttons to enhance interactivity.

### Performance

Performance is really the selling point of `pyqtgraph`. By making use of OpenGL, it can handle huge datasets and still make zooming fast and smooth. This is the fastest animal in the forest.

The performance is often illustrated in examples of real-time signal processing, and this should be kept in mind when designing visualization of streaming data.

### Documentation

The `pyqtgraph` [documentation](http://www.pyqtgraph.org/documentation/) contains plenty of material including installation instructions, examples, as well as a complete API reference.

It is however not clear how up-to-date this documentation is. The copyright note in the page footer gives 2011 as the last update, which is slightly worrying. We have not been able to determine if the footer had not been updated since 2011, or if this applied to the entire documentation.

### Community

There appears to be quite a large number of `pyqtgraph` users, with many examples floating around the web.
Most examples are from private users building a plotting tool for a specific one-time application, usually centered around real-time signal processing. There is no centralized space for collecting examples, as is the case with `matplotlib` or `plotly`.  

### Risks and project sustainability

![pyqtgraph pulse](images/pulse_pyqtgraph.png)

The project however does have some major drawbacks.

#### Lone developer

`pyqtgraph` is only being developed by one person, Luke Campagnola. There are 200+ issues open on github and 100 PRs, including some that are several years old. While it seems many different people are creating PRs, all of them seem to fail the CI tests. However, they are still getting merged regardless. There is a real doubt as to whether this project will still be alive in 5 years.

It is worth mentioning here that the main developer has since joined forces with other OpenGL-based visualization packages to create [`vispy`](http://vispy.org/index.html). See more on `vispy` below.

#### Use in Jupyter has limitations

When using `pyqtgraph` in a Jupyter notebook, one cannot embed the plots in the notebook, a separate `Qt` window has to be spawned. This causes problems if running on a remote machine, as well as making it difficult to remember which plot corresponded to which cell in a lengthy notebook.

In addition, when spawning a `Qt` window via the
```Python
app = QtGui.QApplication([])
QtGui.QApplication.exec_()
```
method, the cell in the notebook is locked and everything below can only be executed once the plot window has been killed, making the use of Jupyter rather pointless. This can however be alleviated via the use of the magic command
```Python
%gui qt
```

#### 3D visualization is not fully developed

While representing scatter plots, surfaces, etc... works very well in 3D, it feels like development was never fully completed. For instance, there are no annotated axes for 3D plots. Only a mesh that serves as a point of reference for the eye is available, with no tick marks nor any labels/annotations. While it may not be too difficult to add, it is not obvious development would be swift, judging by the number of open PRs.

### Standard use cases
See [here](examples/pyqtgraph.ipynb) for a notebook with the `pyqtgraph` standard use cases.

### Summary

<span style="color:green">**Pros:**</span>

- Lightning fast performance
- Very complete in terms of features, including clickable/dragable points/lines
- Good integration with `Qt`
- Handles 1 to 3 dimensions under a single interface

<span style="color:red">**Cons:**</span>

- Only one developer
- Cannot embed plots in Jupyter notebook
- 3D visualization is not fully developed (missing axes)
- No official support from institution or company

---

<a id='bqplot'></a>
## Bqplot (https://github.com/bloomberg/bqplot)

`bqplot` is a relatively new library (started 2015), that is supported by Quantstack and developed by the makers of Jupyter, `ipywidgets`, `ipyvolume`, and `glueviz`. It is targetted at Jupyter users. It is built on `ipywidgets`, and every item in the graphs are a widget, making plots highly configurable with many interaction possibilities (see the author's presentation at [Scipy 2018](https://www.youtube.com/watch?v=Dmxa2Kyfzxk)).

In [ ]:
from bqplot import pyplot as plt

fig1 = plt.figure()
line = plt.plot(x, z[:, 0])
plt.show()
fig2 = plt.figure()
hist = plt.hist(z[:, 0], bins=20)
plt.show()
fig3 = plt.figure()
heatmap = plt.heatmap(z, x=x, y=y)
plt.show()
fig4 = plt.figure()
scat = plt.scatter(z[:, 0], z[:, 1])
plt.show()

### Interactivity

Interactivity is at the heart of `bqplot`.
It is, for example, possible to drag individual points in a scatter plot.

In [ ]:
## Enabling moving of points in scatter. Try to click and drag any of the points in the scatter and 
## notice the line representing the mean of the data update
fig = plt.figure()
scat = plt.scatter(z[:, 0], z[:, 1], colors=['orange'], enable_move=True)
lin = plt.plot([], [], line_style='dotted', colors=['orange'])

def update_line(change=None):
    with lin.hold_sync():
        lin.x = [np.min(scat.x), np.max(scat.x)]
        lin.y = [np.mean(scat.y), np.mean(scat.y)]

update_line()

# update line on change of x or y of scatter
scat.observe(update_line, names=['x'])
scat.observe(update_line, names=['y'])

fig

One can also create linked plots where data selections in one graph can affect the data displayed in another plot. This in fact can be made to interact with any `ipywidgets` based entity, as is illustrated in [this](https://ipyvolume.readthedocs.io/en/latest/bqplot.html#bqplot-scatter-plot) example with `ipyvolume`.

### Performance

Performance is the achilles heel of `bqplot`. The time for rendering graphs is similar to other projects, but interaction feels very slow. Just having 5 histograms on the same plot makes zooming in and out painfully slow: 

In [ ]:
N = 50
M = 5
x = np.arange(N)
y = np.random.rand(N, M)
fig = plt.figure()
for i in range(M):
    plt.bar(x, y[:, i], padding=0.0, color=[i]*N)
plt.show()

It is possible this comes from the fact that every item on the plot (i.e. each bar in the histogram) is a widget of its own, with adjustable properties (one can, for instance, set opacities for individual bars), which would make vectorization of operations difficult. This is pure speculation, and I have no evidence to back this.

There are many issues on the `github` repository on the topic of performance, and this is obviously an area of very active development. In particular it appears that they are trying to make use of WebGL more and more (see [here](https://github.com/glue-viz/bqplot-image-gl)), which would clearly help. Maybe it is simply a matter of time before performance is on par with other similar projects? It is worth coming back to `bqplot` in a year's time to see how this has developed.

### Documentation

Documentation is rather poor on the [readthedocs](https://bqplot.readthedocs.io/en/latest/index.html) website. It basically contains the auto-generated API reference and not much more.

To get to grips with how to use `bqplot`, one has to use the (actually very useful) Jupyter notebooks that are bundled up into a [binder](https://mybinder.org/v2/gh/bloomberg/bqplot/0.11.x?filepath=examples/Index.ipynb) image.

### Community

The `bqplot` community is naturally smaller than that of other projects, due to its relatively young age. Finding examples for `bqplot`, beyond the notebooks in the provided Binder image, can be somewhat difficult. Their [gitter](https://gitter.im/bloomberg/bqplot) space is an additional place to get help, but searching through an un-archived gitter history is not trivial.

### Risks and project sustainability
![bqplot pulse](images/pulse_bqplot.png)

The team of `bqplot` developers comprise, among others, Sylvain Corlay (Quantstack founder) & Maarten Breddels (ipyvolume creator), and they are part of several collaborations that make use of `bqplot`, such as [`glue-jupyter`](https://github.com/glue-viz/glue-jupyter), a Jupyter version of the `glueviz` project by Thomas Robitaille (who is also a core `astropy` contributor).

Development appears to have been steady over the past 5 years, but it also seems to happens in several different places. Some changes are for instance first implemented in `glue-viz/bqplot-image-gl`, before they are integrated in the main repository. This could be both a good and bad thing. On one hand the developement is driven by another application, with frequent requests for new features, and this ensures continous development. On the other hand, there is always the risk that development gets scattered in different directions, no longer having a common goal. This can very well lead to parts of the project becoming disconnected, developing at different speeds, and utlimately break apart. There is no evidence of this yet, but it has to be kept in mind when selecting a framework that is meant to last for a significant part of the lifetime of ESS.

#### Missing features

`bqplot` is currently missing several relatively basic features that other libraries offer. For instance, it is not possible to have error bars on 1D line plots or histograms. One can achieve something that resembles error bars using the `Candles` (OHLC) plots with a zero filled area, but a horizontal bar remains, thus tricking the viewer into thinking these are horizontal error bars.

Another important missing feature is the ability to make either line or filled contour plots. [Some development](https://github.com/glue-viz/bqplot-image-gl/pull/10) is under way in this area, but again it seems like quite a basic feature which is still lacking. The contour plots themselves are not necessarily fundamental for our purposes, but it is more a testimony to the fact that the library is not very mature. The developers have focussed more on fully embracing the widget concept to create a special brand of interactivity, instead of supporting all the possible way of visualizing data, which is a perfectly valid approach. I would simply recommend coming back to `bqplot` in a year's time to see how they have progressed.

### Standard use cases
See [here](examples/bqplot.ipynb) for a notebook with the `bqplot` standard use cases.

### Summary

<span style="color:green">**Pros:**</span>

- Team of developers that are well-known in the python visualization and scientific computing
- Offers uncommon interactivity possibilities, including clickable/dragable points/lines
- Good integration with Jupyter

<span style="color:red">**Cons:**</span>

- Project still feels young and in development, with some relatively basic missing features
- Poor performance on pan/zoom, for even simple plots
- No 3D

---

<a id='bokeh'></a>
## Bokeh

`bokeh` is another very popular plotting library designed to work inside the browser. It is fast, has many interaction tools, and a polished aesthetic. It is in many ways similar to `plotly`, and can do most of the same things.

Some features are missing, such as (filled) contours and 2D image with non-iniform sized pixels.

### Interactivity

Interactivity is best handled through `bokeh`'s own set of widgets. It is very well-rounded, allowing complex handling of selection events and linked plots. However, the interaction is defined through writing javascript code in a long string and passed to a `CustomJS` object (see [here](https://bokeh.pydata.org/en/latest/docs/user_guide/interaction/callbacks.html)).
This is not optimal, and makes interaction with other jupyter widgets difficult.

### Performance

Performance is similar to `plotly`.

### Documentation
Documentation pages are very extensive, well-written and maintained, showcasing a multitude of examples.

### Community

### Risks and project sustainability
![bokeh pulse](images/pulse_bokeh.png)
The project has seen continuous development over the past 6+ years. It is supported by `Numfocus`, so the sustainability risks are very low.

### Standard use cases
See [here](examples/bokeh.ipynb) for a notebook with the `bokeh` standard use cases.

### Summary

<span style="color:green">**Pros:**</span>

- Extensive interaction possibilities
- Well-maintained and well-documented project
- Very complete in terms of features

<span style="color:red">**Cons:**</span>

- No 3D
- Need to write javascript code for interactions with widgets
- Some missing features (contours, non-uniform 2D images)

In short, we did not find a good reason to use `bokeh` over `plotly`. They are very similar in many respects, but `plotly` can do more.

---

<a id='holoviz'></a>
## HoloViz / hvPlot / Holoviews (https://holoviz.org/)

`HoloViz` is an effort to create a unified project that selects the best visualization packages and combines them under a single interface, to try and cover all cases and make the end-user's life easier. It seems like a very difficult task, and what they have achieved is actually pretty impressive. It is worth reading their background [page](), where they explain their motivations, and the sub-projects they are combining to make `HoloViz`.

"[...] we have developed a set of open-source Python packages to streamline the process of working with small and large datasets (from a few datapoints to billions or more) in a web browser, whether doing exploratory analysis, making simple widget-based tools, or building full-featured dashboards. The main libraries in this ecosystem include:

- [Panel](http://panel.pyviz.org/): Assembling objects from many different libraries into a layout or app, whether in a Jupyter notebook or in a standalone servable dashboard

- [Bokeh](http://bokeh.pydata.org/): Interactive plotting in web browsers, running JavaScript but controlled by Python

- [hvPlot](http://hvplot.pyviz.org/): Quickly return interactive Bokeh-based HoloViews or GeoViews objects from Pandas, Xarray, or other data structures

- [HoloViews](http://holoviews.org/): Declarative objects for instantly visualizable data, building Bokeh plots from convenient high-level specifications

- [GeoViews](http://geo.holoviews.org/): Visualizable geographic data that that can be mixed and matched with HoloViews objects

- [Datashader](http://datashader.org/): Rasterizing huge datasets quickly as fixed-size images

- [Param](http://param.pyviz.org/): Declaring user-relevant parameters, making it simple to work with widgets inside and outside of a notebook context

The `Panels` is the equivalent to `plotly`'s `Dash`, and would be extremely useful to create graphical apps. We have not explored creating a `Panels` app, and defer this to a more in-depth evaluation of `HoloViz`.

Overall, our impression of `HoloViz` was extremely positive, and the code in the [standard use cases](examples/holoviz.ipynb) was significantly shorter than for other libraries. You seem to get many things for free in `HoloViz`, which presumably was one of the project's goals, and in that sense one could say they have succeeded. We also very much appreciated how simple it was to switch to the `Datashader` when the data sets were very large.

One drawback of `HoloViz` is that it feels like all possibilities have not quite yet been gathered under a single interface. One often has the choice of either using `hvplot` or `Holoviews` to view datasets, and going to each project's documentation is necessary to understand the syntax and differences between the two.

### Interactivity

Interactivity is very diverse. `HoloViz` appears to be doing for us all the javascript work we did not want to do in `bokeh`, which is a definitely a plus.

### Performance

Default performance is not as high as, for example, `plotly`'s `Scattergl` method. However, by `datashading` the output, one can achieve extremely high performance, as long as it is acceptable to have rasterized outputs.

### Documentation
The individual sub-projects all have very well-written documentation, and there is an obvious and welcome attempt to keep styles uniform across the different websites.
The umbrella `HoloViz` [website](https://holoviz.org) itself does not contain so much material. It does describe the project and its goals in a nice way.

### Community

### Risks and project sustainability

![holoviz pulse](images/pulse_holoviz.png)

The project is relatively new. Up until June 2019, the project was called [`pyviz`](https://pyviz.org/), and was trying to cover an even larger ecosystem of python visualization libraries.
The github pulse shows a longer lifetime but it includes commits from the previous `pyviz` project.

The underlying libraries (`bokeh`, `holoviews`, `datashader`) have been for much longer, and are in no risk of disappearing, but the common `HoloViz` is too young for us to be able to guess whether it would still be alive in 5 years time. It is, at least in part, supported by Anaconda, Quantsight, and NumFocus.

### Standard use cases
See [here](examples/holoviz.ipynb) for a notebook with the `holoviz` standard use cases.

### Summary

<span style="color:green">**Pros:**</span>

- Extensive interaction possibilities
- Well-maintained and well-documented sub-projects
- Hides the javascript from interactions using `bokeh`
- We very much liked the addition of `datashader`
- Also has an app platform `Panels` like `plotly`'s `Dash`
- It can do a lot for you, such as adding sliders automatically to 3D data

<span style="color:red">**Cons:**</span>

- Data has to be in either `pandas` or `xarray` format
- Uses `plotly` for 3D, probably could have used a more performance-oriented solution

---

<a id='ipyvolume'></a>
## Ipyvolume (https://ipyvolume.readthedocs.io/en/latest/)

`ipyvolume` does exactly what it says on the tin: "a Python library to visualize 3d volumes and glyphs (e.g. 3d scatter plots), in the Jupyter notebook, with minimal configuration and effort."

It is probably the fastest way to get a useful 3D interactive plots in a Jupyter notebook. It is based on `ipywidgets`, and uses WebGL via the `pythreejs` library to render in the browser. `pythreejs` is a python interface to the very powerful [`three.js`](https://threejs.org/examples/#webgl_animation_cloth) library for 3D graphics in modern browsers.

As it is based on javascript, plots can be saved as standalone HTML code which can be embedded in any web page.

Many things can be done with `ipyvolume`, as illustrated by one of the author's presentations at [Scipy 2018](https://www.youtube.com/watch?v=hOKa8klJPyo), and by a couple of examples below.

In [ ]:
# A volume rendering plot
import ipyvolume as ipv
N = 128
x = np.linspace(0.0, 2.0, N)
x, y, z = np.meshgrid(x, x, x)
V = np.sin(x**2 + y**2 + z**2)**2
ipv.quickvolshow(V)

In [ ]:
# A scatter plot
x, y, z = np.random.random((3, 5000))
ipv.quickscatter(x, y, z, size=1, marker="sphere")

### Interactivity

Widget interaction is at the heart of `ipyvolume`, where sliders can be used to interact with the graph elements. Markers can be resized, colors can be changed.

It is also possible to link `ipyvolume` to other libraries such as `bqplot` or `bokeh`, whereby a selection in a 2D scatter plot can be highlighted in a 3D `ipyvolume` spatial analog (the [examples](https://ipyvolume.readthedocs.io/en/latest/examples.html) in the documentation show how to achieve this).
This means `ipyvolume` is a natural candidate for complementing a 2D graphics library with a 3D viewer.

### Performance

Performance is impressive. While not at the level of OpenGL solutions like `pyqtgraph`, it surpasses `plotly` and `matplotlib` in terms of smoothness of interaction. It is a 3D tool that focusses on doing 3D well, instead of trying to do everything in a single package.

Interaction with `ipywidgets` is fast, much better than other 3D projects like `pyvista`.

### Documentation

Documentation pages are relatively complete. The examples found there are useful, but few in numbers. The API seems out-dated in some cases. For instance, certain newer input parameters to some functions are missing from the API. The footer states 2017 as the last update, which could explain the discrepancies.

### Community

The community around `ipyvolume` is active in [gitter](https://gitter.im/maartenbreddels/ipyvolume).

### Risks and project sustainability

![ipyvolume pulse](images/pulse_ipyvolume.png)

The project is only 2-3 years old, but development has slowed down a lot over the past year or so. The main developer Maarten Breddels is collaborating on many other projects (`bqplot`, `glueviz`, ...) and `ipyvolume` has been somewhat left behind. This is potentially quite a large risk, as so far it seems that the project will not be picked up if the lead developer leaves.

### Summary

<span style="color:green">**Pros:**</span>

- Great 3D visualization in the Jupyter notebook
- Very good interactivity with widgets and other 2D libraries
- Can save scenes as standalone HTML files
- Interface allows very easy creation of 3D scenes in just a few lines
- Good performance via three.js and WebGL

<span style="color:red">**Cons:**</span>

- Project still feels young and in development
- Lone developer, and project development has been slow for 1+ year.
- Can only plot cubic volumes, aspect ratios are lost by default (hacks can be done to fix this)

---

<a id='other'></a>
## Other projects

Here we list other projects we have taken a glancing look at but have not investigated in detail. Further exploration of these tools would be desirable to ensure we have not overlooked a potentially crucial tool.

### Vispy (http://vispy.org/)

`vispy` was born out of [`galry`](https://github.com/rossant/galry), [`glumpy`](https://glumpy.github.io/), [`visvis`](https://github.com/almarklein/visvis), and [`pyqtgraph`](http://www.pyqtgraph.org/), with the promise of being an extremely high performance graphing library that offloads computations and renderings to the GPU using OpenGL.

It certainly looks promising, from one of their early presentations at [Scipy 2015](https://www.youtube.com/watch?v=_3YoaeoiIFI), but the overall impression when trying to use it is mixed.

The documentation is not very user-friendly, and the gallery of examples is very limited (has been saying "More to come soon!" for as long as I can remember). The examples themselves seem to involve quite complicated code that relates to shaders packed in a string, followed by a lengthy canvas class construction, just to generate [a simple scatter plot](https://github.com/vispy/vispy/blob/master/examples/demo/gloo/cloud.py). It feels like most of this low-level code should be hidden from the user, or at least as a user I would certainly appreaciate if that was the case.

When digging a little deeper, one can actually find examples that look closer to what one would expect from a high-level plotting package, such as a [scatter plot](https://github.com/vispy/vispy/blob/master/examples/basics/plotting/scatter_histogram.py) with histograms on the side axes. These examples **do not** however run in a `IPython` console. One has to run them in a separate python script from the terminal.

Just like `pyqtgraph`, the plots are by default rendered in a `Qt` window. The website does give instructions on how to render them in a Jupyter notebook (by installing an extension), but when trying the scatter histogram examples above I was not able to get the scatter plot to show, only the histograms were visible.

Overall, performance is good but it somehow feels worse that `pyqtgraph`. Zooming in a out on plots feels sluggish, both in the `Qt` window and especially in the Jupyter notebook (and this is on a HP workstation with an Nvidia GPU).

### Altair (https://altair-viz.github.io/)

`Altair` is also capable of linking data in different plot, reacting through selection of points. As shown in [this](https://altair-viz.github.io/gallery/scatter_with_histogram.html) example, one can select a range in a histogram and only the corresponding points inside the selected bins will appear in the scatter plot.

This is an increasingly popular data visualization technique embraced for instance by the `glueviz` project and many others.

One of the main disadvantages of `Altair` is that it requires the data to be in a `pandas` DataFrame, thus either forcing us to use `pandas` as our data container, or requiring a (potentially costly?) conversion step between our data container and `pandas` for the plotting.

### Pyvista (https://docs.pyvista.org/)

`pyvista`, formally known as `vtki`, is a python interface to the VTK library that greatly simplifies the creation of highly-performant 3D visualizations. It is a 3D-specific package, and renders in a separate `Qt` window. Integration in Jupyter notebooks seems possible, but not natively supported. 

### Itkwidgets (https://pypi.org/project/itkwidgets/)

`itkwidgets` is a small project based on the Insight Toolkit (ITK, https://itk.org/), "an open-source, cross-platform system that provides developers with an extensive suite of software tools for image analysis". `itkwidgets` provides a 3D viewer for the Jupyter notebook, with volume rendering and 3-slice renderings.

It uses [`VTK.js`](https://kitware.github.io/vtk-js/index.html), and we found via an [example](https://hub.gke.mybinder.org/user/insightsoftware-tium-itkwidgets-84y4bguw/notebooks/examples/3DImage.ipynb) of a brain structure that `itkwidgets` is one of the best volume renderers out there. 

`itkwidgets` does depend on ITK which is a rather large dependency (~140 MB), which is a downside, unless other parts of ITK can also be used for our purposes.

We also note that the `pyvista` developers are [planning](https://github.com/pyvista/pyvista-support/issues/12#issuecomment-513485873) to use `itkwidgets` to render inside Jupyter notebooks. 

## Non-Python Javascript-based solutions

ParaViewWeb and many many others
